In [14]:
import numpy as np

In [35]:
class Value:
    def __init__(self, data, _children=(), _op=None):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.grad = 0
        self._backward = lambda : None
        
    def __repr__(self):
        return f'Value: {self.data}'
    
    def __add__(self, other):
        out = Value(self.data + other.data,(self, other), _op='+')
        def _backward():
            self.grad += other.data
            other.grad += self.data
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, (self,other), _op='*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def tanh(self):
        x = self.data
        th = (np.exp(2*x) -1) / (np.exp(2*x) + 1)
        out = Value(th, (self,), 'tanh' )
        def _backward():
            self.grad += (1-th**2) * out.grad
        out._backward = _backward
        return out
    
    def backward(self):
        toplogical = []
        visited = set()
        def build_graph(node):
            if node not in visited:
                visited.add(node)
            for n in node._prev:
                build_graph(n)
            toplogical.append(node)
        build_graph(o)
        toplogical = reversed(toplogical)
        for node in toplogical:
            node._backward()
    

In [36]:
x1 = Value(2)
x2 = Value(0)
w1 = Value(-3)
w2 = Value(1)

b = Value(6.7)

x1w1 = x1*w1
x2w2 = x2*w2

x1w1x2w2 = x1w1 + x2w2

print(x1w1x2w2)

n = x1w1x2w2 + b
o = n.tanh()
o.grad = 1

Value: -6


In [37]:
o.backward()